In [78]:
!pip install mlflow dagshub -qqq

In [79]:
import mlflow
import dagshub

dagshub.init(
    repo_name="mlflow_major",  
    repo_owner="iambikash378", 
    mlflow=True               
)

mlflow.set_tracking_uri(f"https://dagshub.com/iambikash378/mlflow_major.mlflow")

Initialized MLflow to track repo "iambikash378/mlflow_major"

Repository iambikash378/mlflow_major initialized!

In [80]:
import os
os.environ["MLFLOW_TRACKING_USERNAME"] = "iambikash378"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "0bc681204678038909d3a123144d2dc73900d017"


In [81]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch import optim, nn
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
from torchvision import models
from torch.nn.functional import relu
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os

## Loss Functions


### Binary Cross Entropy Loss

In [27]:
import torch
import torch.nn.functional as F

def bce_loss(y_pred, y_true, smooth = 1e-6):
    y_pred = y_pred.float().view(-1)
    y_true = y_true.float().view(-1)

    # Binary cross entropy calculation
    bce = - (y_true * torch.log(y_pred + smooth) + (1 - y_true) * torch.log(1 - y_pred + smooth)).mean()

    return bce

    
    

### Dice Loss and Dice Score

In [94]:

def dice_loss(y_pred, y_true, smooth = 1e-6):
    y_pred = y_pred.float().view(-1)
    y_true = y_true.float().view(-1)

    # print(y_pred.shape, y_true.shape)
    # print(y_pred.min(), y_pred.max())  # Should be between 0 and 1 (after sigmoid)
    # print(y_true.min(), y_true.max())  # Should be 0 or 1

    intersection = (y_pred * y_true).sum()
    union = y_pred.sum()+y_true.sum()
    if ((2.0*intersection)+smooth) > (union+smooth):
        print("Error !")
    dice = ((2.0 * intersection) + smooth) / (union + smooth)

    return 1-dice

def dice_score(y_pred, y_true):
    y_pred = torch.sigmoid(y_pred)
    y_pred = (y_pred > 0.5).float()
    intersection = (y_pred * y_true).sum()
    return (2.0 * intersection + 1e-6) / (y_pred.sum() + y_true.sum() + 1e-6) 

### Focal Loss

In [ ]:
import torch
from torch import nn

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.bce = nn.BCEWithLogitsLoss(reduction='none')

    def forward(self, preds, targets):
        bce_loss = self.bce(preds, targets)
        p_t = torch.sigmoid(preds) * targets + (1 - torch.sigmoid(preds)) * (1 - targets)
        focal_weight = self.alpha * (1 - p_t) ** self.gamma
        focal_loss = focal_weight * bce_loss
        return focal_loss.mean()


### Lovasz Loss

In [ ]:
from torch.autograd import Variable

class LovaszHingeLoss(nn.Module):
    def __init__(self):
        super(LovaszHingeLoss, self).__init__()

    def forward(self, preds, targets):
        preds = torch.sigmoid(preds)  # Convert logits to probabilities
        errors = (1 - 2 * targets) * preds  # Compute errors
        sorted_errors, indices = torch.sort(errors, dim=0, descending=True)
        grad = torch.linspace(1, 0, sorted_errors.numel()).to(preds.device)
        loss = torch.dot(F.relu(sorted_errors), grad)
        return loss.mean()

### Dice Loss + BCE Loss (Balances Region Overlap and Pixel Wise Classification)

In [29]:
import torch
import torch.nn as nn

class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, preds, targets):

        preds = torch.sigmoid(preds)

        preds_flat = preds.view(-1)
        targets_flat = targets.view(-1)

        intersection = torch.sum(preds_flat * targets_flat)
        union = torch.sum(preds_flat) + torch.sum(targets_flat)

        dice_score = (2. * intersection + self.smooth) / (union + intersection + self.smooth)

        dice_loss = 1 - dice_score
        return dice_loss


class ComboLoss(nn.Module):
    def __init__(self, bce_weight=0.25, dice_weight=0.75):
        super(ComboLoss, self).__init__()
        self.bce = nn.BCEWithLogitsLoss()
        self.dice = DiceLoss()
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight

    def forward(self, preds, targets):
        bce_loss = self.bce(preds, targets)
        dice_loss = self.dice(preds, targets)
        return self.bce_weight * bce_loss + self.dice_weight * dice_loss

### Lovasz + Dice Loss

In [183]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def dice_loss(pred, target, smooth=1e-6):
    """Computes Dice loss."""
    pred = pred.view(-1)  # Flatten
    target = target.view(-1)  # Flatten
    
    intersection = (pred * target).sum()
    dice = (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)
    
    return 1 - dice  # Dice loss (1 - Dice coefficient)

def lovasz_hinge(logits, labels):
    """Lovász hinge loss for binary segmentation."""
    def lovasz_grad(gt_sorted):
        """Computes gradient of Lovász extension."""
        gts = gt_sorted.sum()
        intersection = gts - gt_sorted.float().cumsum(0)
        union = gts + (1 - gt_sorted).float().cumsum(0)
        jaccard = 1. - intersection / union
        jaccard[1:] = jaccard[1:] - jaccard[:-1]
        return jaccard

    logits = logits.view(-1)
    labels = labels.view(-1)

    signs = 2. * labels - 1.  # Convert {0,1} labels to {-1,1}
    errors = 1. - logits * signs
    errors_sorted, perm = torch.sort(errors, descending=True)
    gt_sorted = labels[perm]
    
    return torch.dot(F.relu(errors_sorted), lovasz_grad(gt_sorted))

class CombinedLoss(nn.Module):
    """Combined Lovász + Dice Loss for highly imbalanced segmentation."""
    def __init__(self, dice_weight=0.5, lovasz_weight=0.5):
        super().__init__()
        self.dice_weight = dice_weight
        self.lovasz_weight = lovasz_weight

    def forward(self, logits, targets):
        probs = torch.sigmoid(logits)  # Convert logits to probabilities

        dice = dice_loss(probs, targets)
        lovasz = lovasz_hinge(logits, targets)  # Use logits directly for Lovász hinge loss

        return self.dice_weight * dice + self.lovasz_weight * lovasz

# Usage in training loop:
criterion = CombinedLoss(dice_weight=0.5, lovasz_weight=0.5)

# Example usage:
logits = torch.randn(2, 1, 128, 128)  # Example model output
targets = torch.randint(0, 2, (2, 1, 128, 128)).float()  # Binary ground truth mask

loss = criterion(logits, targets)
print("Loss:", loss.item())


Loss: 0.9681923389434814


## Dataset Loader


In [184]:

class hrgldd_dataset(Dataset): 
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)
      
    def __getitem__(self, index):
        img = self.x[index]
        label = self.y[index]

        img = torch.from_numpy(img).float().permute(2,0,1)
        label = torch.from_numpy(label).float().permute(2,0,1)

        return img, label

path_to_testX = '/kaggle/input/hrgldd-data/testX.npy'
path_to_testY = '/kaggle/input/hrgldd-data/testY.npy'
path_to_trainX = '/kaggle/input/augmented-training/jointX.npy'
path_to_trainY = '/kaggle/input/augmented-training/jointY.npy'
path_to_valX = '/kaggle/input/hrgldd-data/valX.npy'
path_to_valY = '/kaggle/input/hrgldd-data/valY.npy'

data_testY = np.load(path_to_testY)
data_testX = np.load(path_to_testX)
data_trainX = np.load(path_to_trainX)
data_trainY = np.load(path_to_trainY)
data_valX = np.load(path_to_valX)
data_valY = np.load(path_to_valY)

print(data_trainY.shape)
print(data_trainX.shape)
print(data_trainX.dtype)
print(data_trainY.dtype)
print(data_valX.shape)
print(data_valY.shape)


train_dataset = hrgldd_dataset(data_trainX, data_trainY)
im, lbl = train_dataset[0]

print(im.shape)
#print(lbl)
print(max(lbl))

(3357, 128, 128, 1)
(3357, 128, 128, 4)
float32
float32
(284, 128, 128, 4)
(284, 128, 128, 1)
torch.Size([4, 128, 128])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


## Dataset Loader fo OnTheFlyAugmentation only 3 Bands

In [165]:
# path_to_testX_otf = '/kaggle/input/hrgldd-data/testX.npy'
# path_to_testY_otf = '/kaggle/input/hrgldd-data/testY.npy'
# path_to_trainX_otf = '/kaggle/input/hrgldd-data/trainX.npy'
# path_to_trainY_otf = '/kaggle/input/hrgldd-data/trainY.npy'
# path_to_valX_otf = '/kaggle/input/hrgldd-data/valX.npy'
# path_to_valY_otf = '/kaggle/input/hrgldd-data/valY.npy'

# data_testY_otf = np.load(path_to_testY)
# data_testX_otf = np.load(path_to_testX)
# data_trainX_otf = np.load(path_to_trainX)
# data_trainY_otf = np.load(path_to_trainY)
# data_valX_otf = np.load(path_to_valX)
# data_valY_otf = np.load(path_to_valY)


# import torch
# from torch.utils.data import Dataset
# from torchvision import transforms
# import random
# from PIL import Image

# class HRGLDDataset_onthefly(Dataset):
#     def __init__(self, x, y, transform=None, label_transform=None):
#         self.x = x  # BGR + NIR images
#         self.y = y  # Labels (e.g., masks)
#         self.transform = transform  # Optional transformation for images
#         self.label_transform = label_transform  # Optional transformation for labels

#     def __len__(self):
#         return len(self.x)

#     def __getitem__(self, index):
#         img = self.x[index]
#         label = self.y[index]

#         img = img[..., :3]

#         img = np.clip(img, 0, 1) * 255  # If the image is normalized, clip and scale
#         img = img.astype(np.uint8)  # Convert to uint8 (necessary for PIL)
#         # Convert images to PIL Image (since many transforms work on PIL images)
#         img = Image.fromarray(img[..., ::-1])

#         label = label.squeeze()  # Remove any unnecessary dimensions, making it 2D (H, W)
#         # Convert label to uint8 if it's a binary mask
#         label = (label * 255).astype(np.uint8)  # Scale to 0 or 255 for masks
#         label = Image.fromarray(label)  # Assuming label is also in numpy format, convert to PIL

#         # Apply image transformations (color jitter, flipping, rotation, etc.)
#         if self.transform:
#             img = self.transform(img)  
        
#         if self.label_transform:
#             label = self.label_transform(label)  # Apply transformations to the label (mask)

#         # Convert to tensor and permute to match (C, H, W) format for PyTorch
#         img = torch.from_numpy(np.array(img)).float().permute(2, 0, 1)
#         img = img/255.0# BGR + NIR to CxHxW
        
        
#         label = torch.from_numpy(np.array(label)).float().unsqueeze(0)  # Mask to CxHxW format (1 channel)
#         label = label/255.0

#         return img, label

# # Define image transformation pipeline (for BGR + NIR)
# image_transform = transforms.Compose([
#     transforms.RandomHorizontalFlip(),  # Random horizontal flip
#     transforms.RandomVerticalFlip(),    # Random vertical flip
#     transforms.RandomRotation(30),      # Random rotation between -30 to 30 degrees
#     transforms.RandomResizedCrop(128, scale=(0.8, 1.0)),  # Random cropping and scaling (crop size: 128x128)
#     transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Random color jitter
# ])

# # Define label transformation pipeline (spatial transformations)
# label_transform = transforms.Compose([
#     transforms.RandomHorizontalFlip(),  # Random horizontal flip for the label
#     transforms.RandomVerticalFlip(),    # Random vertical flip for the label
#     transforms.RandomRotation(30),      # Random rotation for the label
#     transforms.RandomResizedCrop(128, scale=(0.8, 1.0)),  # Random cropping and scaling for the label
# ])

# # Instantiate the dataset with both image and label transformations
# train_dataset_otf = HRGLDDataset_onthefly(data_trainX_otf, data_trainY_otf, transform=image_transform, label_transform=label_transform)
# val_dataset_otf = hrgldd_dataset(data_valX_otf, data_valY_otf)

# i, l = train_dataset_otf[100]
# print(torch.max(l))
# print(len(train_dataset))

tensor(1.)
3357


## Model (UNET)

In [185]:


class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(out_channels, out_channels,kernel_size = 3, padding = 1),
            nn.ReLU(inplace = True)
        )

    def forward(self, x):
        return self.conv(x)
    

class DownSample(nn.Module):
    
    def __init__(self, in_channels, out_channels):
        super(DownSample, self).__init__()
        self.conv = DoubleConv(in_channels, out_channels)
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)

    def forward(self,x):
        down = self.conv(x)
        p = self.pool(down)
        return down, p


class UpSample(nn.Module):
    
    def __init__(self,in_channels,out_channels):
        super(UpSample,self).__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels//2,
                                     kernel_size = 2,
                                     stride = 2)
        self.conv = DoubleConv(in_channels,out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        x = torch.cat([x1,x2],dim = 1)
        x2 = self.conv(x)
        return x2
    
    
class UNet(nn.Module):

    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        self.down_conv_1 = DownSample(in_channels, 64)
        self.down_conv_2 = DownSample(64, 128)
        self.down_conv_3 = DownSample(128, 256) 
        self.down_conv_4 = DownSample(256, 512)

        self.bottleneck = DoubleConv(512, 1024)

        self.up_conv_1 = UpSample(1024,512)
        self.up_conv_2 = UpSample(512,256)
        self.up_conv_3 = UpSample(256,128)
        self.up_conv_4 = UpSample(128,64)

        self.out = nn.Conv2d(in_channels=64, out_channels = 1,
                             kernel_size = 3, padding = 1
                             )

    def forward(self,x):
        #print(f"Input Shape : {x.shape}")
        
        down1, p1 = self.down_conv_1(x)
        #print(f"Shape after doubl_conv_1_only : {down1.shape}")
        #print(f"Shape after down_conv_1 : {p1.shape}")
        
        down2, p2 = self.down_conv_2(p1)
        #print(f"Shape after doubl_conv_2_only : {down2.shape}")
        #print(f"Shape after down_conv_2: {p2.shape}")
        
        down3, p3 = self.down_conv_3(p2)
        #print(f"Shape after doubl_conv_3_only : {down3.shape}")
        #print(f"Shape after down_conv_3 : {p3.shape}")
        
        down4, p4 = self.down_conv_4(p3)
        #print(f"Shape after doubl_conv_4_only : {down4.shape}")
        #print(f"Shape after down_conv_4 : {p4.shape}")

        b = self.bottleneck(p4)
        #print(f"Shape after bottleneck : {b.shape}")

        up_1 = self.up_conv_1(b, down4)
        #print(f"Shape after up_1 : {up_1.shape}")
        up_2 = self.up_conv_2(up_1,down3)
        #print(f"Shape after up_2 : {up_2.shape}")
        up_3 = self.up_conv_3(up_2, down2)
        #print(f"Shape after up_3 : {up_3.shape}")
        up_4 = self.up_conv_4(up_3, down1)
        #print(f"Shape after up_4 : {up_4.shape}")

        op = self.out(up_4)
        #print(f"Shape of output : {op.shape}")

        return op

    


## Train

### Training Loop for UNet with 64 Filters

In [ ]:
LEARNING_RATE = 5e-4
BATCH_SIZE = 32
EPOCHS = 500
PATIENCE = 20
DATA_PATH = "/kaggle/input/hrgldd-data"
MODEL_SAVE_PATH = "/kaggle/working/best_model.pth"

model_dir = os.path.dirname(MODEL_SAVE_PATH)
if not os.path.exists(model_dir):
    print(f"Creating directory: {model_dir}")
    os.makedirs(model_dir, exist_ok=True)
else:
    print(f"Directory already exists: {model_dir}")

val_loss_list = []
train_loss_list = []
f1_score_list = []
recall_list = []
accuracy_list = []
precision_list = []

criterion = CombinedLoss(dice_weight=0.5, lovasz_weight=0.5)


train_dataset = hrgldd_dataset(data_trainX, data_trainY)
val_dataset = hrgldd_dataset(data_valX, data_valY)
test_dataset = hrgldd_dataset(data_testX, data_testY)

device = "cuda" if torch.cuda.is_available() else "cpu"
train_dataloader = DataLoader(dataset=train_dataset,
                               batch_size=BATCH_SIZE,
                               shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=True)

model = UNet(4, 1).to(device)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

best_val_loss = float('inf')

with mlflow.start_run():
    mlflow.log_params(
        {
            "initial_learning_rate": LEARNING_RATE,
            "batch_size": BATCH_SIZE,
            "model_arch": 'UNet',
            "loss_function": 'combined lov dice',
            "epochs": EPOCHS,
            "patience": PATIENCE,
            "filters": [64, 128, 256, 512, 1024],
            "optimizer": type(optimizer).__name__,
            "threshold" : 0.5,
            "remarks" : "augmented data"
        }
    )

    for epoch in range(EPOCHS):
        train_running_loss = 0
        model.train()
        for index, x_y in enumerate(train_dataloader):
            img = x_y[0].float().to(device)
            mask = x_y[1].float().to(device)

            optimizer.zero_grad()
            outputs = model(img)
            train_loss = criterion(outputs, mask)
            train_loss.backward()  # Find Gradients by Backward Pass
            optimizer.step()  # Update the weights

            train_running_loss += train_loss.item()

        train_loss = train_running_loss / len(train_dataloader)
        mlflow.log_metric("train_loss", train_loss, step=epoch)

        train_loss_list.append(train_loss)

        # --------------- Validation Part ------------------------ #

        model.eval()
        val_running_loss = 0
        #val_dice = 0
        val_accuracy = 0
        val_precision = 0
        val_recall = 0
        val_f1 = 0
        with torch.no_grad():
            for index, x_y in enumerate(val_dataloader):
                img = x_y[0].float().to(device)
                mask = x_y[1].float().to(device)

                y_pred = model(img)
                val_loss = criterion(y_pred, mask)
                val_running_loss += val_loss.item()


                y_pred_binary = (torch.sigmoid(y_pred) > 0.6).float()
                mask_binary = (mask > 0.5).float()
                y_pred_flat = y_pred_binary.view(-1).cpu().numpy()
                mask_flat = mask_binary.view(-1).cpu().numpy()

                val_accuracy += accuracy_score(mask_flat, y_pred_flat)
                val_precision += precision_score(mask_flat, y_pred_flat, zero_division=0)
                val_recall += recall_score(mask_flat, y_pred_flat, zero_division=0)
                val_f1 += f1_score(mask_flat, y_pred_flat, zero_division=0)

            val_loss = val_running_loss / len(val_dataloader)
            val_loss_list.append(val_loss)
            #val_dice = val_dice / len(val_dataloader)
            val_accuracy = val_accuracy / len(val_dataloader)
            accuracy_list.append(val_accuracy)
            val_precision = val_precision / len(val_dataloader)
            precision_list.append(val_precision)
            val_recall = val_recall / len(val_dataloader)
            recall_list.append(val_recall)
            val_f1 = val_f1 / len(val_dataloader)
            f1_score_list.append(val_f1)

            mlflow.log_metric("val_loss", val_loss, step=epoch)
            mlflow.log_metric("val_accuracy", val_accuracy, step=epoch)
            mlflow.log_metric("val_precision", val_precision, step=epoch)
            mlflow.log_metric("val_recall", val_recall, step=epoch)
            mlflow.log_metric("val_f1", val_f1, step=epoch)

        print("--" * 30)
        print(f"Train Loss EPOCH {epoch + 1}: {train_loss:.4f}")
        print(f"Val Loss EPOCH {epoch + 1} : {val_loss:.4f}")
        #print(f"Val Dice Score EPOCH {epoch + 1} : {val_dice:.4f}")
        print(f"Val Precision EPOCH {epoch + 1}: {val_precision:.4f}")
        print(f"Val Recall EPOCH {epoch + 1}: {val_recall:.4f}")
        print(f"Val F1 Score EPOCH {epoch + 1}: {val_f1:.4f}")
        print(f"Current Learning Rate : {optimizer.param_groups[0]['lr']:.6f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), MODEL_SAVE_PATH)
            mlflow.log_artifact(MODEL_SAVE_PATH)
            print(f"Saved best model with val loss : {val_loss:.4f}")
        else:
            counter += 1
            print(f"No improvement in validation loss for {counter} epochs.")

        if counter >= PATIENCE:
            print(f"Early stopping triggered after {PATIENCE} epochs without improvement.")
            break

        scheduler.step(val_loss)


In [8]:
model = UNet(in_channels=4, out_channels=1).to(device)

x = torch.randn(16, 4, 128, 128).to(device)  # Batch of 16 images, 4 channels, 256x256 resolution
output = model(x)
print(output.shape)

torch.Size([16, 1, 128, 128])
